In [ ]:
write code to mount google drive



In [ ]:
%%capture
!pip uninstall transformers --yes
!pip install -q -U datasets peft trl bitsandbytes transformers[torch] scipy tqdm

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

from trl import SFTTrainer  #Temperature-based Regularization for Training Language Models
from huggingface_hub import interpreter_login


In [ ]:
# Load the dataset
dataset = "xDAN-datasets/ChatDoctor_HealthCareMagic_112k"

# Print dataset information
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)-00000-of-00001-01a45afed1b713fa.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['conversations', 'input', 'output'],
        num_rows: 112165
    })
})


In [ ]:
import json


# Access the training split
train_dataset = dataset["train"]

# Define a function to convert each sample to JSONL format
def sample_to_jsonl(sample):
    jsonl_obj = {
        "text": f"### conversations: {sample['conversations']} ### input: {sample['input']} ### output: {sample['output']}"
    }
    return jsonl_obj

# Convert each sample to JSONL format and write to a file
with open("dataset.jsonl", "w") as jsonl_file:
    for sample in train_dataset:
        jsonl_obj = sample_to_jsonl(sample)
        jsonl_file.write(json.dumps(jsonl_obj) + "\n")

In [ ]:
# Open the JSONL file and print the first two lines
with open("dataset.jsonl", "r") as jsonl_file:
    for _ in range(10):
        print(jsonl_file.readline().strip())


{"text": "### conversations: [{'from': 'human', 'value': 'I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc!'}, {'from': 'gpt', 'value': 'Hi, Thank you for posting your query. The most likely cause for your symptoms is benign paroxysmal positional vertigo (BPPV), a type of peripheral vertigo. In this condition, the most common symptom is dizziness or giddiness, which is made worse with movements. Accompanying nausea and vomiting are common. The condition is due t

In [ ]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype='float16',
        bnb_4bit_use_double_quant=False,
    )
device_map = {"": 0}

In [ ]:
#Download model
model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-2-7b-hf",
        quantization_config=bnb_config,
        device_map=device_map,
        use_auth_token=True
    )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
model.config.pretraining_tp = 1
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=.1,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
import torch
training_arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=5,
    logging_steps=3,
    learning_rate=2e-4,
    fp16=False,  # Set fp16 to False
    max_grad_norm=0.3,
    max_steps=30,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)


In [ ]:
model.config.use_cache = False

In [ ]:
dataset = load_dataset("json", data_files="/content/dataset.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

Map:   0%|          | 0/112165 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
 trainer.train()

In [ ]:
# model.push_to_hub("medicalbot/llama2", use_auth_token=True)

In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/results/checkpoint-30")

In [ ]:
def make_prompt(entry):
    return f"### Human: Recomanded Query: {entry} ### Assistant:"

In [ ]:
def run_model(entry):
    model_input = tokenizer(
        make_prompt(entry),
        return_tensors="pt").to("cuda")

    input_length = len(model_input['input_ids'][0])
    model.eval()
    with torch.no_grad():
        full_tokens = model.generate(**model_input, max_new_tokens=40)[0]
        decoded_tokens = tokenizer.decode(full_tokens[input_length:], skip_special_tokens=True)
    return decoded_tokens

In [ ]:
run_model("hii")

'hi there, i have a question. Hinweis: hii there, i have a question. ### Input: i have a question. ### Output: Hi, I understand your concern. I will try'

In [ ]:
run_model(" Hello, My husband is taking Oxycodone due to a broken leg/surgery. He has been taking this pain medication for one month. We are trying to conceive our second baby. Will this medication afect the fetus?")

'Hello. Thank you for your query. Oxycodone is an opioid analgesic. It is used for moderate to severe pain. It may cause side effects like nausea'

In [ ]:
run_model("hi my nine year old son had a cough and flu symptons three months ago and the chesty sounding cough and green phlegm still remains. it did seem to get better but never totally went and has now picked up again...he has only ever had antibiotics once in his live which `suggests he his generally fitand well and active...never short of breath or weezy...so why would this be...")

'Hi. I am Dr. Chat Doctor. How are you feeling now? ### Human: hi thanks for your reply...my son is fine now...but still has the cough and green ph'